In [4]:
import importlib

from amlta.question_generation import generate, prepare_train

generate = importlib.reload(generate)
prepare_train = importlib.reload(prepare_train)

In [2]:
r = generate.generate_random_process_question()
r_parsed = r.choices[0].message.parsed
if r_parsed:
    print(r_parsed.model_dump_json(indent=2))

<process>
Name: Kohle-Kessel-Karibik

Year:
- '2000'

Geography: CAB

Class:
- Energieversorgung / Fernwärmeversorgung

Technology:
- |+
  Kohlekessel für Prozesswärme, ohne besondere Emissionsminderung, alle Daten nach #1
  Auslastung: 4500h/a
  Brenn-/Einsatzstoff: Brennstoffe-fossil-Kohle
  gesicherte Leistung: 100%
  Jahr: 50
  Lebensdauer: 25a
  Leistung: 50MW
  Nutzungsgrad: 85%
  Produkt: Wärme - Prozess

Main Output:
- 1 TJ  Prozesswärme
- Main Output Flow: Flow("process heat" output, 1e+06 MJ (Net calorific value), type='Product flow', class='Systems / Other systems')
</process>

<query_params>
query_type: names
direction: input
aggregation: sum
flow_names:
- waste
</query_params>

<flows_result>
Flows:
  Inputs: 1
  Outputs: 0
  Input Flows:
  - Flow("waste" input, 164 MJ (Net calorific value), type='Waste flow', class='End-of-life treatment / Other end-of-life services')
  Output Flows: []
</flows_result>

# Results
The above results are purely for reference. The question sh

In [25]:
batch_in_path = generate.prepare_batch(
    n=200,
    offset=200,
    questions_per_process=3,
    model="gpt-4o-mini",
)

In [18]:
batch = generate.send_batch(batch_in_path)

In [19]:
batch

Batch(id='batch_67b0aa9fb96c8190bf58d07cdc9f0e2f', completion_window='24h', created_at=1739631263, endpoint='/v1/chat/completions', input_file_id='file-FzgFWPY9gzhmfTqURZK3RG', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1739717663, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [20]:
batch_out_path = generate.retrieve_batch_results(batch.id)

Batch(id='batch_67b0aa9fb96c8190bf58d07cdc9f0e2f', completion_window='24h', created_at=1739631263, endpoint='/v1/chat/completions', input_file_id='file-FzgFWPY9gzhmfTqURZK3RG', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='token_limit_exceeded', line=None, message='Enqueued token limit reached for gpt-4o-mini in organization org-MmVXbqt4t9VTzg8dGuXtgBBf. Limit: 2,000,000 enqueued tokens. Please try again once some in_progress batches have been completed.', param=None)], object='list'), expired_at=None, expires_at=1739717663, failed_at=1739631264, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


RuntimeError: Batch not completed: Errors(data=[BatchError(code='token_limit_exceeded', line=None, message='Enqueued token limit reached for gpt-4o-mini in organization org-MmVXbqt4t9VTzg8dGuXtgBBf. Limit: 2,000,000 enqueued tokens. Please try again once some in_progress batches have been completed.', param=None)], object='list')

In [5]:
prepare_train.process_batch_results(batch_out_path)

Saved 600 questions to D:\dev\amlta-project\data\generated\questions\out\batch_gpt-4o-mini_0_200.jsonl


[{'batch': 'batch_gpt-4o-mini_0_200',
  'process_uuid': '404ee608-dcb1-4c7e-9bda-b5eafe763547',
  'question_id': 0,
  'basic_query': 'energy supply in Germany',
  'general_query': 'solar electricity transmission process',
  'specific_query': 'electricity import from solar energy in Germany 2030',
  'flow_query_params': {'query_type': 'class',
   'direction': 'output',
   'aggregation': 'list',
   'flow_class': 'Emissions'},
  'question': 'What are the total emissions to air from <the process>?',
  'question_replaced_basic': 'What are the total emissions to air from energy supply in Germany?',
  'question_replaced_general': 'What are the total emissions to air from solar electricity transmission process?',
  'question_replaced_specific': 'What are the total emissions to air from electricity import from solar energy in Germany 2030?'},
 {'batch': 'batch_gpt-4o-mini_0_200',
  'process_uuid': '404ee608-dcb1-4c7e-9bda-b5eafe763547',
  'question_id': 1,
  'basic_query': 'electricity supply p